## Work
1. 請自行定義一個 loss function, 為 0.3 * focal loss + 0.7 cross-entropy，訓練並比較結果

請參考範例程式碼Day089_CustomizedLoss.ipynb，請自行定義一個 loss function, 為 0.3 * focal loss + 0.7 cross-entropy，訓練並比較結果 (請讓比例為可以隨意調控，而非寫死)



In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 10
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [42]:
import tensorflow as tf
import keras.backend as K

"""Code Here
撰寫一個 loss function, 使其可以結合 focal loss 與 crossentropy loss
"""
#crossentropy loss: keras.losses.categorical_crossentropy(y_true, y_pred)


def combined_loss(gamma=2., alpha=4., crossentropy_precentage=0):
    """Define the customized loss."""  
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        """
        #裡面的操作都要用tensor來做
        epsilon = 1e-8
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
            
        #cl = keras.losses.categorical_crossentropy(y_true, y_pred)
            
        #return (1. - crossentropy_precentage) * tf.reduce_mean(reduced_fl) + (crossentropy_precentage * cl)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed
    
ce_weights_list = [0., 0.3, 0.5, 0.7, 1]

In [43]:
import itertools
results = {}

for i, ce_w in enumerate(ce_weights_list):
    print("Numbers of exp: %i, ce_weight: %.2f" % (i, ce_w))

    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    """Code Here
    將自定義的 loss function 加入模型
    """
    model.compile(loss=combined_loss(crossentropy_precentage=ce_w), metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True
             )
    
    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, ce_weight: 0.00
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_58 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_59 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_60 (Batc (None, 128) 

W0805 13:44:31.245641 4762387904 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1206: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 26s 513us/step - loss: 5.9919 - acc: 0.3336 - val_loss: 6.1888 - val_acc: 0.3344
Epoch 2/10
50000/50000 [==============================] - 11s 212us/step - loss: 4.3788 - acc: 0.4450 - val_loss: 4.6976 - val_acc: 0.4150
Epoch 3/10
50000/50000 [==============================] - 9s 186us/step - loss: 3.9782 - acc: 0.4821 - val_loss: 4.4158 - val_acc: 0.4277
Epoch 4/10
50000/50000 [==============================] - 13s 256us/step - loss: 3.7007 - acc: 0.5103 - val_loss: 4.1972 - val_acc: 0.4556
Epoch 5/10
50000/50000 [==============================] - 9s 176us/step - loss: 3.4899 - acc: 0.5283 - val_loss: 4.1690 - val_acc: 0.4602
Epoch 6/10
50000/50000 [==============================] - 8s 168us/step - loss: 3.3124 - acc: 0.5506 - val_loss: 4.2990 - val_acc: 0.4387
Epoch 7/10
50000/50000 [==============================] - 10s 200us/step - loss: 3.1380 - acc: 0.5670 - val_loss: 4.022

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 10s 209us/step - loss: 6.0942 - acc: 0.3292 - val_loss: 6.1134 - val_acc: 0.3467
Epoch 2/10
50000/50000 [==============================] - 8s 152us/step - loss: 4.3590 - acc: 0.4450 - val_loss: 4.8045 - val_acc: 0.4104
Epoch 3/10
50000/50000 [==============================] - 9s 175us/step - loss: 3.9502 - acc: 0.4852 - val_loss: 4.6025 - val_acc: 0.4238
Epoch 4/10
50000/50000 [==============================] - 8s 157us/step - loss: 3.6855 - acc: 0.5125 - val_loss: 4.3345 - val_acc: 0.4421
Epoch 5/10
50000/50000 [==============================] - 8s 159us/step - loss: 3.4928 - acc: 0.5306 - val_loss: 4.3576 - val_acc: 0.4326
Epoch 6/10
50000/50000 [==============================] - 8s 162us/step - loss: 3.3008 - acc: 0.5507 - val_loss: 4.0651 - val_acc: 0.4655
Epoch 7/10
50000/50000 [==============================] - 9s 172us/step - loss: 3.1369 - acc: 0.5663 - val_loss: 4.0644 -

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()